In [ ]:
# 1️⃣  Clean up and reinstall missing dependencies
!pip install -U triton==2.3.0 bitsandbytes==0.43.1 accelerate transformers codecarbon

# 2️⃣  Restart runtime (important!)
import os
os.kill(os.getpid(), 9)


INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s et

In [ ]:
import torch, bitsandbytes as bnb, triton

print("✅ CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0))
print("BitsAndBytes version:", bnb.__version__)
print("Triton version:", triton.__version__)
print("Torch version:", torch.__version__)


✅ CUDA available: True
CUDA device: Tesla T4
BitsAndBytes version: 0.43.1
Triton version: 2.3.0
Torch version: 2.3.1+cu121


In [ ]:
!pip install -q -U transformers==4.41.2 accelerate==0.30.1 bitsandbytes==0.43.1 codecarbon==3.0.7


In [ ]:
import transformers, bitsandbytes, torch
print("Transformers:", transformers.__version__)
print("BitsAndBytes:", bitsandbytes.__version__)
print("Torch:", torch.__version__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Transformers: 4.41.2
BitsAndBytes: 0.43.1
Torch: 2.3.1+cu121


In [ ]:
# ⚡ QUANTIZED MODEL BLOCK (8-bit, FINAL STABLE VERSION)
from codecarbon import EmissionsTracker
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch, time, csv, os
from datetime import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

model_name = "deepseek-ai/deepseek-coder-1.3b-base"
output_path = "/content/results"
os.makedirs(output_path, exist_ok=True)
csv_path = os.path.join(output_path, "quantized_results.csv")

code_sample = """from typing import List
def below_zero(operations: List[int]) -> bool:
    balance = 0
    for op in operations:
        balance += op
        if balance < 0:
            return True
    return False
"""

bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

print("Loading model in 8-bit mode...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
).eval()

print("✅ Model loaded successfully!")

tracker = EmissionsTracker(output_dir=output_path, project_name="Quantized_Run")
tracker.start()

latencies = []
for epoch in range(1, 6):
    start = time.time()
    inputs = tokenizer(code_sample, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64)
    latency = time.time() - start
    latencies.append(latency)
    print(f"Epoch {epoch}: {latency:.2f}s")

total_emission = tracker.stop()
avg_latency = sum(latencies) / len(latencies)
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Model", "Quantization", "Avg_Latency_s", "CO2_kg", "Timestamp"])
    writer.writerow(["DeepSeek", "8-bit", avg_latency, total_emission, timestamp])

print(f"\n✅ Quantized run complete! Results saved to:\n{csv_path}")
print(f"⏱️ Avg Latency: {avg_latency:.2f}s | 🌱 CO₂ Emission: {total_emission:.6f} kg")


Running on: cuda
Loading model in 8-bit mode...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

[codecarbon WARNING @ 18:12:49] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:12:49] [setup] RAM Tracking...
[codecarbon INFO @ 18:12:49] [setup] CPU Tracking...


...

Epoch 3: 8.62s


[codecarbon INFO @ 18:13:36] Energy consumed for RAM : 0.000125 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:13:36] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:13:36] Energy consumed for All CPU : 0.000531 kWh
[codecarbon INFO @ 18:13:36] Energy consumed for all GPUs : 0.000367 kWh. Total GPU Power : 29.805541689698202 W
[codecarbon INFO @ 18:13:36] 0.001023 kWh of electricity used since the beginning.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Epoch 4: 10.35s


[codecarbon INFO @ 18:13:48] Energy consumed for RAM : 0.000157 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:13:48] Delta energy consumed for CPU with constant : 0.000137 kWh, power : 42.5 W
[codecarbon INFO @ 18:13:48] Energy consumed for All CPU : 0.000668 kWh
[codecarbon INFO @ 18:13:48] Energy consumed for all GPUs : 0.000464 kWh. Total GPU Power : 30.085966778031974 W
[codecarbon INFO @ 18:13:48] 0.001289 kWh of electricity used since the beginning.


Epoch 5: 9.41s

✅ Quantized run complete! Results saved to:
/content/results/quantized_results.csv
⏱️ Avg Latency: 11.32s | 🌱 CO₂ Emission: 0.000179 kg


In [ ]:
!pip uninstall -y transformers bitsandbytes accelerate
!pip install transformers==4.39.3 bitsandbytes==0.43.0 accelerate==0.27.2 codecarbon


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 124.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
ERROR: pip's dependency resolver does not curre

In [ ]:
import torch, bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
print("✅ CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")
print("BitsAndBytes version:", bnb.__version__)


✅ CUDA available: True
CUDA device: Tesla T4
BitsAndBytes version: 0.43.0


In [ ]:
# ===============================================================
# ⚡ QUANTIZED MODEL BLOCK (8-bit, FINAL FULL VERSION)
# ===============================================================

from codecarbon import EmissionsTracker
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch, time, csv, os
from datetime import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

model_name = "deepseek-ai/deepseek-coder-1.3b-base"
output_path = "/content/results"
os.makedirs(output_path, exist_ok=True)
csv_path = os.path.join(output_path, "quantized_results.csv")

# ===============================================================
# 🧩 SAMPLE INPUT + PROMPT
# ===============================================================
code_sample = """from typing import List
def below_zero(operations: List[int]) -> bool:
    balance = 0
    for op in operations:
        balance += op
        if balance < 0:
            return True
    return False
"""

prompt_text = (
    "You are an expert Python developer. "
    "Given the function below, generate a complete and well-formatted Google-style docstring. "
    "Include purpose, parameters, returns, and one example. "
    "Provide only the full function code with the docstring.\n\n"
    f"{code_sample}"
)

# ===============================================================
# ⚙️ HELPER FUNCTIONS
# ===============================================================
def evaluate_accuracy(text):
    checks = ['"""', "Args:", "Returns:"]
    return "P" if all(x in text for x in checks) else "F"

def get_model_size_mb(model):
    torch.save(model.state_dict(), "temp.pt")
    size = os.path.getsize("temp.pt") / (1024 * 1024)
    os.remove("temp.pt")
    return round(size, 2)

# ===============================================================
# 🧠 LOAD MODEL (8-bit)
# ===============================================================
bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

print("Loading model in 8-bit mode...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
).eval()

print("✅ Model loaded successfully!")

# ===============================================================
# 🌱 EMISSION TRACKER + EXPERIMENT LOOP
# ===============================================================
EPOCHS = 5
tracker = EmissionsTracker(output_dir=output_path, project_name="Quantized_Run")

# prepare CSV
if not os.path.exists(csv_path):
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Model", "Quantization", "Epoch", "CO2_kg",
            "Latency_s", "Model_Size_MB", "Accuracy", "Timestamp"
        ])

# main loop
for epoch in range(1, EPOCHS + 1):
    print(f"\n⚡ Epoch {epoch}/{EPOCHS}...")
    tracker.start()

    start = time.perf_counter()
    inputs = tokenizer(prompt_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.3,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    latency = round(time.perf_counter() - start, 3)
    co2 = tracker.stop()

    text = tokenizer.decode(output[0], skip_special_tokens=True)
    acc = evaluate_accuracy(text)
    size = get_model_size_mb(model)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    with open(csv_path, "a", newline="") as f:
        csv.writer(f).writerow(
            ["DeepSeek", "8-bit", epoch, co2, latency, size, acc, timestamp]
        )

print(f"\n✅ Quantized experiment complete. Results saved to: {csv_path}")


Running on: cuda
Loading model in 8-bit mode...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: Fut

✅ Model loaded successfully!


[codecarbon WARNING @ 18:38:00] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:38:00] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:38:00] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 18:38:00] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:38:00] [setup] GPU Tracking...
[codecarbon INFO @ 18:38:00] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:38:00] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 18:38:00] >>> Tracker's metadata:
[codecarbon INFO @ 18:38:00]   Platform sys


⚡ Epoch 1/5...


...


⚡ Epoch 4/5...


/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
[codecarbon WARNING @ 18:40:40] Tracker already stopped !
[codecarbon INFO @ 18:40:40] Energy consumed for RAM : 0.000444 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:40:40] Delta energy consumed for CPU with constant : 0.000489 kWh, power : 42.5 W
[codecarbon INFO @ 18:40:40] Energy consumed for All CPU : 0.001887 kWh
[codecarbon INFO @ 18:40:40] Energy consumed for all GPUs : 0.001413 kWh. Total GPU Power : 32.40991416814337 W
[codecarbon INFO @ 18:40:40] 0.003744 kWh of electricity used since the beginning.
[codecarbon WARNING @ 18:40:53] Already started tracking



⚡ Epoch 5/5...


/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
[codecarbon WARNING @ 18:41:23] Tracker already stopped !
[codecarbon INFO @ 18:41:23] Energy consumed for RAM : 0.000563 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:41:23] Delta energy consumed for CPU with constant : 0.000505 kWh, power : 42.5 W
[codecarbon INFO @ 18:41:23] Energy consumed for All CPU : 0.002392 kWh
[codecarbon INFO @ 18:41:23] Energy consumed for all GPUs : 0.001804 kWh. Total GPU Power : 32.890197267914346 W
[codecarbon INFO @ 18:41:23] 0.004758 kWh of electricity used since the beginning.



✅ Quantized experiment complete. Results saved to: /content/results/quantized_results.csv
